In [1]:
using JuMP
using Ipopt
using LinearAlgebra

In [2]:
model = Model(Ipopt.Optimizer)
n = 32
@variable(model, 50 ≤ D[1:n]≤ 2000)
@variable(model, 90 ≤ H[1:n] ≤ 110)
Fa = 10 
L = [88; 400; 88; 100; 350;72.23; 55.17 ;308.49; 275.88 ;248.08; 186.48; 2319.52; 578.64; 1852.22; 1519.05; 429.81; 1482.04; 1458.17; 2219.7; 716.42; 447; 895; 447; 894; 2683.19; 1788.81; 2722.91; 1341.62; 447.19; 1341.56; 894; 894.44] 
P(x) = 0.128x^2-5.0331x+544.19
CustoDiam(D...)= sum(L[i]*P(D[i]) for i in 1:n)
CustoAltura(H...) = sum(H[i]*L[i]*9.8*Fa for i in 1:n)

JuMP.register(model, :CustoDiam, n, CustoDiam, autodiff=true)
JuMP.register(model, :CustoAltura, n, CustoAltura, autodiff=true)

@NLobjective(model, Min, CustoDiam(D...) + CustoAltura(H...) + sum(5e4*((1.325/log((0.025/3.7*D[i])+(5.74/(2*D[i])^0.9)))*(L[i]/D[i])*(H[i]/19.6)) for i in 1:n) )

In [3]:
for i in 1:n
    @NLconstraint(model, 5 ≤ ((1/1e9)*3600*2*(H[i]/100)*π*(D[i]/2)^2) ≤ 25)
    @NLconstraint(model, 10e-6 ≤ 0.025/D[i] ≤ 10e-2)
    @NLconstraint(model, 500 ≤ 2*D[i] ≤ 3000)
end
@NLconstraint(model,0 ≤ CustoDiam(D...) + CustoAltura(H...) + sum(5e4*((1.325/log((0.025/3.7*D[i])+(5.74/(2*D[i])^0.9)))*(L[i]/D[i])*(H[i]/19.6)) for i in 1:n) ≤ 22e9)

0 <= CustoDiam(D[1], D[2], D[3], D[4], D[5], D[6], D[7], D[8], D[9], D[10], D[11], D[12], D[13], D[14], D[15], D[16], D[17], D[18], D[19], D[20], D[21], D[22], D[23], D[24], D[25], D[26], D[27], D[28], D[29], D[30], D[31], D[32]) + CustoAltura(H[1], H[2], H[3], H[4], H[5], H[6], H[7], H[8], H[9], H[10], H[11], H[12], H[13], H[14], H[15], H[16], H[17], H[18], H[19], H[20], H[21], H[22], H[23], H[24], H[25], H[26], H[27], H[28], H[29], H[30], H[31], H[32]) + (50000.0 * ((1.325 / log((0.025 / 3.7) * D[1] + 5.74 / (2.0 * D[1]) ^ 0.9)) * (88.0 / D[1]) * (H[1] / 19.6)) + 50000.0 * ((1.325 / log((0.025 / 3.7) * D[2] + 5.74 / (2.0 * D[2]) ^ 0.9)) * (400.0 / D[2]) * (H[2] / 19.6)) + 50000.0 * ((1.325 / log((0.025 / 3.7) * D[3] + 5.74 / (2.0 * D[3]) ^ 0.9)) * (88.0 / D[3]) * (H[3] / 19.6)) + 50000.0 * ((1.325 / log((0.025 / 3.7) * D[4] + 5.74 / (2.0 * D[4]) ^ 0.9)) * (100.0 / D[4]) * (H[4] / 19.6)) + 50000.0 * ((1.325 / log((0.025 / 3.7) * D[5] + 5.74 / (2.0 * D[5]) ^ 0.9)) * (350.0 / D[5]) * (H

In [4]:
model

A JuMP Model
Minimization problem with:
Variables: 64
Objective function type: Nonlinear
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 64 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 64 constraints
Nonlinear: 97 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: D, H

In [5]:
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:      192
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       64
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       64
                     variables with only upper bounds:        0
Total number of equa

In [6]:
objective_value(model)

3.24783579699358e9

In [7]:
value.(D)

32-element Array{Float64,1}:
 896.5560929591674
 896.5560929591662
 896.5560929591674
 896.5560929591672
 896.5560929591662
 896.5560929591677
 896.5560929591684
 896.5560929591663
 896.5560929591663
 896.5560929591663
 896.5560929591667
 896.556092959166
 896.5560929591662
   ⋮
 896.5560929591662
 896.556092959166
 896.5560929591662
 896.556092959166
 896.556092959166
 896.556092959166
 896.556092959166
 896.556092959166
 896.5560929591661
 896.5560929591661
 896.556092959166
 896.556092959166

In [8]:
value.(H)

32-element Array{Float64,1}:
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
   ⋮
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0
 110.0